<a href="https://colab.research.google.com/github/ucfilho/ANN_raianars/blob/master/ANN_004_gera_matriz_classifica_OUT_18_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# este comando clona o diretorio github ucfilho/ANN no colaboratory research
!git clone https://github.com/RaianaSeixas/ANN #clonar do Github
%cd ANN
#!ls
print("Hello world"+"\n"+"I'm evolving!!!"+"\n")

%matplotlib inline

Cloning into 'ANN'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 76 (delta 1), reused 0 (delta 0), pack-reused 70
Unpacking objects: 100% (76/76), done.
/content/ANN
Hello world
I'm evolving!!!



In [2]:
!ls

000_ANN_Gera_Matriz_out_04_2019.ipynb		    X_DAT_B_set_17_2019.csv
000_ANN_parte_001_XY_dados_out_01_2019.ipynb	    X_DAT_set_17_2019.csv
000_ANN_parte_002_XY_Métricas_D10_OUT_4_2019.ipynb  XY_D10_30_out_7_2019.csv
000_ANN_parte_009_Métricas_PCA_OUT_07_2019.ipynb    XY_D10_out_1_2019.csv
000_Gerenciador_DE_var_F_CR_STAT_set_27_2019.ipynb  XY_D10_out_5_2019.csv
imput_ANN_D10_30_out_07_19.zip			    XY_schwefel_out_11_19.csv
imput_ANN_D10_6F.zip				    Y_DAT_B_set_17_2019.csv
imput_ANN_D10_out_05_19.zip			    Y_DAT_set_17_2019.csv
imput_ANN_schwefel_out_11_19.zip


In [0]:
# Functions
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

def tasig(x,deriv=False):
    if(deriv==True):
        return 1-x*x
    return 2/(1+np.exp(-2*x))-1

# purelin function
def purelin(x,deriv=False):
    if(deriv==True):
        return 1
    #N=len(x)
    #for i in range(N):
      #if (x[i]>1):
        #x[i]=1
    return x

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
def ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,X_train,y):
  syn0 = 2*np.random.random((Neuron_input,Neuron_hidden)) - 1
  syn1 = 2*np.random.random((Neuron_hidden,Neuron_output)) - 1
  for j in range(Epoch):
    l0 = X_train
    l1 = tasig(np.dot(l0,syn0))
    l2 = purelin(np.dot(l1,syn1))
    l2_error = y - l2
    l2_delta = l2_error*purelin(l2,deriv=True)
    l1_error = l2_delta.dot(syn1.T)
    l1_delta = l1_error * tasig(l1,deriv=True)
    syn1 += l1.T.dot(l2_delta)*Learning_rate
    syn0 += l0.T.dot(l1_delta)*Learning_rate
  return syn0,syn1



In [0]:
def ANN_ycal(syn0,syn1,X_train):
  l0 = X_train
  l1 = tasig(np.dot(l0,syn0))
  l2 = purelin(np.dot(l1,syn1))
  y_calc=np.reshape(l2,len(l2))
  return y_calc

In [0]:
def Classifica(ycalc):
  n=len(ycalc)
  for i in range(n):
    if(ycalc[i]<0):
      ycalc[i]=-1
    else:
      ycalc[i]=1
  return ycalc

In [0]:
def PrecisaoGlobal(tn, fp, fn, tp):
  PrecGlob=(tn+tp)/(tn+ fp+ fn+ tp)
  return PrecGlob

In [0]:
# leitura dos dados
DAT=pd.read_csv("XY_D10_30_out_7_2019.csv")
#print(DAT.head())
#print(len(DAT))

In [0]:
# separa as colunas de interesse
Index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)
#print(Dat_Select)
#print(Dat_Select.head())

In [0]:
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [0]:
ColF=DAT_OLD['F']
W=np.array(DAT_OLD['F'])
Wo=np.array(DAT_OLD['Fo'])
for i in range(len(W)):
  W[i]=W[i]-Wo[i]
  if(W[i]<0):
    W[i]=-1
  else:
    W[i]=1
DAT['F'] = W

In [14]:
# obtem as variaveis independentes (treino e teste)
# separa os dados em treino e teste
train=DAT.sample(frac=0.8,random_state=None) 
test=DAT.drop(train.index)
IND=[1,2,3,4,6,7,8,9,10,11,12,13,14]
x_train=train.iloc[:,IND]
x_test=test.iloc[:,IND]
X_OLD=DAT_OLD.iloc[:,IND]
X_NOME=DAT_OLD.columns[IND]
print(X_NOME)


Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')


In [0]:
MELHOR_PRECISAO=0
MAT0=[]
MAT1=[]
CONFUSAO=[]

In [0]:
# obtem as variaveis dependentes (treino e teste)
Index=[15] #F
#Index=[16] #CR
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
#print(y_train.head())

In [0]:
y_train=np.array(y_train)
Nomes=x_train.columns
x_test=np.array(x_test)
x_train=np.array(x_train)

In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.001
omega=0.9
Epoch=20000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_calc_test=ANN_ycal(syn0,syn1,x_test)

In [19]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)
print("====================================")
y_calc_train = Classifica(y_calc_train)
y_calc_test = Classifica(y_calc_test)



tn, fp, fn, tp = confusion_matrix(y_calc_train,y_obs_train).ravel()
PrecTreino=PrecisaoGlobal(tn, fp, fn, tp)
print("Treino precisao global=",PrecTreino)
print("Treino tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_train,y_obs_train))


print("====================================")
tn, fp, fn, tp = confusion_matrix(y_calc_test,y_obs_test).ravel()
PrecTeste=PrecisaoGlobal(tn, fp, fn, tp)
print("Teste: precisao global=",PrecTeste)
print("Teste tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_test,y_obs_test))



if MELHOR_PRECISAO< PrecTeste:
  MELHOR_PRECISAO=PrecTeste
  MAT0=np.copy(syn0)
  MAT1=np.copy(syn1)
  CONF_teste=np.copy(confusion_matrix(y_calc_test,y_obs_test))
  CONF_treino=np.copy(confusion_matrix(y_calc_train,y_obs_train))


Epocas= 20000 Learning rate= 0.001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')
Treino precisao global= 1.0
Treino tn: 58  fp: 0 fn: 0 tp: 43
[[58  0]
 [ 0 43]]
Teste: precisao global= 0.68
Teste tn: 7  fp: 7 fn: 1 tp: 10
[[ 7  7]
 [ 1 10]]


**So para lembrar q o .ravel() existe!!**

In [20]:
print(confusion_matrix(y_calc_test,y_obs_test))
print(confusion_matrix(y_calc_test,y_obs_test).ravel())
a,b,c,d,e,f,g,h=[1,2,3,4,5,6,7,99]
print(h)

[[ 7  7]
 [ 1 10]]
[ 7  7  1 10]
99


In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.0001
omega=0.9
Epoch=100000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_obs_test=y_test
y_calc_test=ANN_ycal(syn0,syn1,x_test)



In [22]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)
print("====================================")
y_calc_train = Classifica(y_calc_train)
y_calc_test = Classifica(y_calc_test)



tn, fp, fn, tp = confusion_matrix(y_calc_train,y_obs_train).ravel()
PrecTreino=PrecisaoGlobal(tn, fp, fn, tp)
print("Treino precisao global=",PrecTreino)
print("Treino tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_train,y_obs_train))


print("====================================")
tn, fp, fn, tp = confusion_matrix(y_calc_test,y_obs_test).ravel()
PrecTeste=PrecisaoGlobal(tn, fp, fn, tp)
print("Teste: precisao global=",PrecTeste)
print("Teste tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_test,y_obs_test))



if MELHOR_PRECISAO< PrecTeste:
  MELHOR_PRECISAO=PrecTeste
  MAT0=np.copy(syn0)
  MAT1=np.copy(syn1)
  CONF_teste=np.copy(confusion_matrix(y_calc_test,y_obs_test))
  CONF_treino=np.copy(confusion_matrix(y_calc_train,y_obs_train))

Epocas= 100000 Learning rate= 0.0001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')
Treino precisao global= 1.0
Treino tn: 58  fp: 0 fn: 0 tp: 43
[[58  0]
 [ 0 43]]
Teste: precisao global= 0.76
Teste tn: 8  fp: 6 fn: 0 tp: 11
[[ 8  6]
 [ 0 11]]


In [0]:
# print(X_NOME)

In [24]:
# ========= Novas mudancas ==================================================
IND=[1,2,6,7]
x_train=train.iloc[:,IND]
x_test=test.iloc[:,IND]
X_OLD=DAT_OLD.iloc[:,IND]
X_NOME=DAT_OLD.columns[IND]
print(X_NOME)
y_train=np.array(y_train)
x_test=np.array(x_test)
x_train=np.array(x_train)

Index(['DI RELATIVO', 'FRAC Its', 'Fo', 'CRo'], dtype='object')


In [25]:
x_train

array([[-9.43263667e-01,  1.00000000e+00,  9.12737992e-01,
        -5.83764907e-01],
       [-9.43263667e-01,  1.00000000e+00, -8.78552849e-01,
         8.79564052e-01],
       [-3.08092060e-01, -9.00004166e-01, -7.93579282e-01,
         1.36123239e-01],
       [-7.86232829e-01,  5.99933336e-01, -1.00000000e+00,
        -1.17122113e-01],
       [ 1.41943079e-01, -3.00029165e-01,  3.96648493e-01,
        -9.59436663e-01],
       [-4.29310671e-01,  7.49927086e-01,  3.17393748e-01,
        -9.02410155e-01],
       [-9.08458215e-01,  4.99937503e-01, -8.77128228e-02,
        -6.22819913e-01],
       [ 3.57231182e-01, -3.00029165e-01, -6.88236030e-01,
        -6.52067646e-01],
       [-4.38477433e-01, -3.00029165e-01, -8.19734128e-02,
        -6.71996197e-01],
       [-9.74237882e-01, -4.16649306e-05, -4.02858291e-01,
        -7.70993688e-02],
       [-9.39218425e-01,  7.49927086e-01, -7.24252110e-01,
        -5.05317683e-01],
       [ 8.77772226e-01,  1.00000000e+00,  3.04727387e-01,
      

In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.0001
omega=0.9
Epoch=300000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_obs_test=y_test
y_calc_test=ANN_ycal(syn0,syn1,x_test)



In [27]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)
print("====================================")
y_calc_train = Classifica(y_calc_train)
y_calc_test = Classifica(y_calc_test)



tn, fp, fn, tp = confusion_matrix(y_calc_train,y_obs_train).ravel()
PrecTreino=PrecisaoGlobal(tn, fp, fn, tp)
print("Treino precisao global=",PrecTreino)
print("Treino tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_train,y_obs_train))


print("====================================")
tn, fp, fn, tp = confusion_matrix(y_calc_test,y_obs_test).ravel()
PrecTeste=PrecisaoGlobal(tn, fp, fn, tp)
print("Teste: precisao global=",PrecTeste)
print("Teste tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_test,y_obs_test))



if MELHOR_PRECISAO< PrecTeste:
  MELHOR_PRECISAO=PrecTeste
  MAT0=np.copy(syn0)
  MAT1=np.copy(syn1)
  CONF_teste=np.copy(confusion_matrix(y_calc_test,y_obs_test))
  CONF_treino=np.copy(confusion_matrix(y_calc_train,y_obs_train))



Epocas= 300000 Learning rate= 0.0001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'Fo', 'CRo'], dtype='object')
Treino precisao global= 0.9108910891089109
Treino tn: 51  fp: 2 fn: 7 tp: 41
[[51  2]
 [ 7 41]]
Teste: precisao global= 0.64
Teste tn: 6  fp: 7 fn: 2 tp: 10
[[ 6  7]
 [ 2 10]]


In [28]:
print("Melhor precisao=",MELHOR_PRECISAO)
print(MAT0)
print(MAT1)
nl0=MAT0.shape[0];nc0=MAT0.shape[1]
nl1=MAT1.shape[0];nc1=MAT1.shape[1]
print("colunas %d e linhas %d MAT0" % (nc0,nl0))
print("colunas %d e linhas %d MAT1" % (nc1,nl1))
print(CONF_teste)
print(CONF_treino)

Melhor precisao= 0.76
[[ 0.98248057 -0.03066945  0.09738595  0.39286539  0.80333667 -1.77166566
  -1.83728932 -0.74829362 -0.11859996]
 [-0.48783853  0.87608532 -0.78921473 -0.60619098 -1.31903574 -0.74646992
   2.747822    0.87376071 -0.35547972]
 [ 1.50180593  0.16632648 -1.60513763 -1.47604956  0.14039004 -1.68171164
  -0.43015861  0.30697764  3.88732203]
 [-0.71477097 -0.38527887 -0.63976298 -0.36106028 -0.72672363 -0.62372047
   1.01701949 -1.19862967 -1.72444324]
 [ 0.47636017  0.73559577 -0.88694046  0.87428399 -0.03359192 -0.90387984
  -2.70309321  1.0592587   1.79449268]
 [ 0.26103078 -0.77078736 -1.09700005  0.85028701  0.45121855 -0.4952592
  -0.51644847  1.59860553  0.76193834]
 [ 0.37160424  0.26270281 -0.33229412 -0.38011197 -0.21202535 -1.03595383
   0.66968145  0.30535212 -1.02564991]
 [-1.03306089  0.72014572 -0.39501507 -0.45114721 -0.24419947 -0.36527481
  -0.22335263 -0.14907577  0.04176141]
 [ 0.50638285  1.30898537  0.03741387  1.73236199 -0.38030668 -0.14127536
 

In [29]:
Nomes=['OTIMO','PARTIC_best','Xresp']
print("Melhor precisao=",MELHOR_PRECISAO)
#print(MAT0)
#print(MAT1)
nl0=MAT0.shape[0];nc0=MAT0.shape[1]
nl1=MAT1.shape[0];nc1=MAT1.shape[1]
print("colunas %d e linhas %d MAT0" % (nc0,nl0))
print("colunas %d e linhas %d MAT1" % (nc1,nl1))
print(CONF_teste)
print(CONF_treino)
#BUSCA=pd.Series([OTIMO,PARTIC_best,Xresp],index=Nomes )

Melhor precisao= 0.76
colunas 9 e linhas 13 MAT0
colunas 1 e linhas 9 MAT1
[[ 8  6]
 [ 0 11]]
[[58  0]
 [ 0 43]]


In [30]:
dim=np.array([[nl0,nc0],[nl1,nc1]])
Dummies=['hello pandas I am envolving']
Armazenar=['Dummie','MAT0','MAT1','dim','Conf_treino','Conf_teste','precisao']
BUSCA=pd.Series([Dummies,MAT0,MAT1,dim,CONF_treino,CONF_teste,MELHOR_PRECISAO],index=Armazenar )
BUSCA
BUSCA = pd.DataFrame(BUSCA) 
print(BUSCA)


                                                             0
Dummie                           [hello pandas I am envolving]
MAT0         [[0.9824805721673776, -0.030669451996601215, 0...
MAT1         [[3.7710607595723142], [1.3883846008012921], [...
dim                                          [[13, 9], [9, 1]]
Conf_treino                                 [[58, 0], [0, 43]]
Conf_teste                                   [[8, 6], [0, 11]]
precisao                                                  0.76


In [33]:
print(BUSCA)

Conf_treino    [[58, 0], [0, 43]]
Conf_teste      [[8, 6], [0, 11]]
precisao                     0.76
dtype: object


In [38]:
cn=nc0+nc1;print(cn)
lmax=max([nl0,nl1]);print(lmax)
print(syn0.shape);print(syn1.shape)
syn0=pd.DataFrame(syn0)
syn1=pd.DataFrame(syn1)
M=np.zeros((lmax,1))
M[0,0]=nl0;M[1,0]=nc0;M[2,0]=nl1;M[3,0]=nc1;
M=pd.DataFrame(M)
for i in range(nc0):
  M[i+1]=syn0[i]
print(M)
for i in range(nc1):
  M[i+nc0+1]=syn1[i]
print(M)
#M[range(1,nc0+1)]=syn0
Armazenar=['CONF_treino','CONF_teste','MELHOR_PRECISAO']
Complemento=np.zeros(lmax-3)
BUSCA=list(['CONF_treino',CONF_treino,'CONF_teste',CONF_teste,'MELHOR_PRECISAO',MELHOR_PRECISAO] )
M[nc0+nc1+2]=pd.Series(BUSCA)
print(M)

10
13
(4, 9)
(9, 1)
       0         1         2         3         4         5         6  \
0   13.0  0.112906 -0.712444 -4.772594 -1.129842 -2.159858  0.239086   
1    9.0  0.226775 -4.082946 -2.159133 -4.576445 -0.757495  0.119021   
2    9.0 -1.015955 -2.726522 -2.903935 -1.608120 -3.085365  2.587146   
3    1.0  0.824789  3.673251  2.403771  5.032467  0.565747  0.234986   
4    0.0       NaN       NaN       NaN       NaN       NaN       NaN   
5    0.0       NaN       NaN       NaN       NaN       NaN       NaN   
6    0.0       NaN       NaN       NaN       NaN       NaN       NaN   
7    0.0       NaN       NaN       NaN       NaN       NaN       NaN   
8    0.0       NaN       NaN       NaN       NaN       NaN       NaN   
9    0.0       NaN       NaN       NaN       NaN       NaN       NaN   
10   0.0       NaN       NaN       NaN       NaN       NaN       NaN   
11   0.0       NaN       NaN       NaN       NaN       NaN       NaN   
12   0.0       NaN       NaN       NaN      

In [0]:
np.zeros((lmax,1))

In [37]:
BUSCA=list(['CONF_treino',CONF_treino,'CONF_teste',CONF_teste,'MELHOR_PRECISAO',MELHOR_PRECISAO] )
print(BUSCA)

['CONF_treino', array([[58,  0],
       [ 0, 43]]), 'CONF_teste', array([[ 8,  6],
       [ 0, 11]]), 'MELHOR_PRECISAO', 0.76]


In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
from datetime import datetime
nowfname=datetime.now().strftime("%Y-%m-%d_%Hh%Mmin")

In [0]:
BUSCA.to_csv(('ANN_'+ nowfname +'.csv'))
print(('ANN_'+ nowfname +'.csv'))
ARQ='ANN_'+ nowfname +'.csv'

In [0]:
#!cp 2019-10-05_03h42min_DE_Michalewicz_10.csv drive/My\ Drive/imput_ANN/
!cp  ANN_2019-10-18_20h20min.csv drive/My\ Drive/

In [0]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

In [0]:
ANN=pd.read_csv('ANN_2019-10-18_19h35min.csv')

In [0]:
print(type(ANN))

In [0]:
print(ANN)

In [0]:
A=np.array(ANN.iloc[0][1])
B=np.array(ANN.iloc[1][1])
C=np.array(ANN.iloc[2][1])
print(A)
print(B)
print(C)